In [ ]:
data_path = "/kaggle/input/fashion-product-images-small/myntradataset"
print(os.listdir(data_path))

In [ ]:
import pandas as pd
import os
import numpy as np

In [ ]:
data = pd.read_csv(os.path.join(data_path,'styles.csv'),error_bad_lines=False, nrows=5000)
data.head()

In [ ]:
data['image'] = data.apply(lambda row: str(row['id']) + ".jpg", axis=1)
data = data.sample(frac=1).reset_index(drop=True)
data.head(10)

In [ ]:
import cv2
test = cv2.imread(os.path.join(data_path,'images','15970.jpg'))
test.shape

In [ ]:
batch_size = 32
from keras_preprocessing.image import ImageDataGenerator

image_generator = ImageDataGenerator(
    validation_split=0.20,
    rescale=1.0/255.0,
     horizontal_flip=True
)

training_generator = image_generator.flow_from_dataframe(
    dataframe=data,
    directory=data_path + "/images",
    x_col="image",
    y_col="articleType",
    target_size=(64,64),
    batch_size=batch_size,
    subset="training"
)

validation_generator = image_generator.flow_from_dataframe(
    dataframe=data,
    directory=data_path + "/images",
    x_col="image",
    y_col="articleType",
    target_size=(64,64),
    batch_size=batch_size,
    subset="validation"
)

classes = len(training_generator.class_indices)

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D,Dense,MaxPool2D,BatchNormalization,Flatten,Dropout,AveragePooling2D,Embedding,LSTM

In [ ]:
def model(width, height, channels, num_classes):
    model = Sequential()
    model.add(Conv2D(6, (7, 7), activation='relu', input_shape=(width, height, channels)))
    model.add(MaxPool2D())
    model.add(Conv2D(16, (5, 5), activation='relu'))
    model.add(MaxPool2D())
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
width ,height, channels, num_classes = 64,64,3,108
mod = model(width ,height, channels, num_classes)

In [ ]:
mod.fit_generator(
    generator=training_generator,
    steps_per_epoch=len(training_generator),

    validation_data=validation_generator,
    validation_steps=len(validation_generator),

    epochs=5
)

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
x = [str(i) for i in data['productDisplayName']]
y = [i for i in data['articleType']]
tokenizer = Tokenizer(num_words=10**4, oov_token='<OOV>')
tokenizer.fit_on_texts(x)
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1
txt_seqs = tokenizer.texts_to_sequences(x)
txt_seqs = pad_sequences(txt_seqs, maxlen=10,
                                 truncating='post', padding='post')
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(y)
y = pd.get_dummies(y)
print(txt_seqs)
X_train, X_test, y_train, y_test = train_test_split(txt_seqs, y, test_size=0.20, random_state=42)

In [ ]:
y_train.shape

In [ ]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(vocab_size, embedding_vecor_length, input_length=10))
model.add(LSTM(100))
model.add(Dense(108, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=64)